# 1. Cel/ Zakres
- Klasteryzacja
- Znajdywanie parametrów dla algorytmów klasteryzacji.

In [1]:
import pandas as pd
from sklearn.datasets import fetch_openml
import numpy as np
import datetime
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

import pickle

# 2. Przygotowanie Danych

In [2]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)
X = mnist["data"]
y = mnist["target"]

# 3. Ćwiczenie

Pozyskane dane (zmienna X) reprezentują zeskanowane znaki nieznanego alfabetu.

Celem ćwiczenia jest identyfikacja ile tych znaków jest i jak mogą one wyglądać.

Zakładając, że możemy mieć do czynienia z 8–12 różnymi znakami uzyj metody centroidów do ich
klasteryzacji.

### 1. Przeprowadź klasteryzację dla 8, 9, 10, 11 i 12 skupisk.

In [3]:
clusters = [8, 9, 10, 11, 12]
silhouette_scores = [] 

for k in clusters:
    kmeans = KMeans(n_clusters = k, random_state=42)
    y_pred = kmeans.fit_predict(X)
    score = silhouette_score(X, kmeans.labels_)
    silhouette_scores.append(score)

### 2. Wylicz wartośc wskaźnika sylwetkowego dla każdego z ww. skupisk.
Zapisz wartość wszystkich wskaźników sylwetkowych jako listę w pliku Pickle o nazwie:
- kmeans_sil.pkl.

In [4]:
with open("kmeans_sil.pkl", 'wb') as f:
    pickle.dump(silhouette_scores, f)

### 3. Wartości wskaźnika sylwetkowego.
Znany lingwista prof. Talent twierdzi, że w zbiorze X można zidentyfikować 10 różnych
znaków. <br /> Czy wartości wskaźnika sylwetkowego potwierdzają tą obserwację?


### 4. Policz macierz błędów.
Prof. Talent dostarczył swoich wyników klasyfikacji w postaci zbioru y. Policz macierz błędów
pomiędzy danymi otrzymanymi z procesu klasteryazacji dla 10 skupisk i zbioru y.


In [5]:
kmeans_10 = KMeans(n_clusters = 10, random_state=42)
y_pred10 = kmeans_10.fit_predict(X)

conf_matrix10 = confusion_matrix(y, y_pred10)

In [6]:
conf_matrix10 

array([[ 290,    2, 1265,   39,    9,    7,  162,    4,   72, 5053],
       [   8, 4293,    7,    7,   10,   11,    7, 3526,    8,    0],
       [ 323,  423,  246,  216, 4863,   78,  147,  436,  201,   57],
       [4581,  449,  461,  193,  216,   45,   31,   58, 1083,   24],
       [   0,  178,  288, 3728,   29, 2173,  168,  234,   17,    9],
       [2129,  155, 1812,  432,    7,  215,   67,  280, 1156,   60],
       [  38,  190, 2068,   67,   53,    4, 4326,   45,   14,   71],
       [   6,  372,   12, 2094,   53, 4399,    4,  314,   18,   21],
       [1212,  335,  292,  208,   53,  193,   51,  330, 4115,   36],
       [  87,  261,   31, 3462,   19, 2849,   16,   95,   87,   51]])

### 5. Dla każdego wiersza ww. macierzy znajdź indeks o najwyższej wartości
(np. numpy.argmax() albo pandas.Series.argmax()).<br/> Wartości umieść na posortowanej rosnąco liście bez duplikatów (użyj np. set()). <br/>Listę zapisz w pliku Pickle o nazwie:
- kmeans_argmax.pkl.

In [7]:
kmeans_argmax = []

for row in conf_matrix10:
    index = np.argmax(row)
    if index not in kmeans_argmax:
        kmeans_argmax.append(index)

In [8]:
kmeans_argmax = np.sort(kmeans_argmax)

In [9]:
with open("kmeans_argmax.pkl", 'wb') as f:
    pickle.dump(kmeans_argmax, f)

### 6. Znajdź sensowne wartości parametru eps dla DBSCAN.
Heurystyka dla określenia wartości parametru eps oparta jest o odległość eulidesową pomiędzy instancjami. Policz odległości dla pierwszych 300 elementów ze zbioru X. <br/>(użyj np. numpy.linalg.norm(x1-x2), gdzie x1 i x2 to punkty w przestrzeni wielowymiarowej), <br/>a następnie wyświetl 10 najmniejszych. Ww.<br/><br/>
10 wartości umieść na liście w kolejności rosnącej, a listę zapisz w pliku Pickle o nazwie:
- dist.pkl.

In [10]:
norms = []

for i in range(300):
    for j in range(len(X)):
        if i == j:
            continue
        norm = np.linalg.norm(X[i] - X[j]) 
        norms.append(norm)

In [11]:
norm_sorted = np.unique(np.sort(norms))
dist = norm_sorted[:10]

In [12]:
with open("dist.pkl", 'wb') as f:
    pickle.dump(dist, f)

### 7. Policz średnią s z 3 najmniejszych wartości z ww. listy.
Przyjmij kolejno wartości eps od s do s+10%*s z krokiem co 4%*s i wykonaj klasteryzacje.


In [13]:
s = np.mean(dist[:3])

### 8. Dla każdej klasteryzacji (dla kolejnych wartości eps) policz ile jest unikalnych etykie zidentyfikowanych przez algorytm DBSCAN.
Wartości umieść na liście i zapisz w pliku Pickle o nazwie:
- dbscan_len.pkl.


In [15]:
eps_values = np.arange(s, 1.1*s, step=0.04*s)
dbscan_len = []

for e in eps_values:
    dbscan = DBSCAN(eps=e)
    dbscan.fit(X)
    unique_labels = set(dbscan.labels_)
    dbscan_len.append(len(unique_labels))
    

In [16]:
dbscan_len

[4, 7, 22]

In [17]:
dbscan_len = "dbscan_len.pkl"

with open(dbscan_len, 'wb') as f:
    pickle.dump(dbscan_len, f)

# 4. Prześlij raport

Prześlij plik o nazwie:
- lab7.py
realizujący ww. ćwiczenia.

Sprawdzane będzie, czy skrypt Pythona tworzy wszystkie wymagane pliki oraz czy ich zawartość jest poprawna.